In [1]:
import sys
import os
curdir=sys.path[0]
pardir=os.path.abspath(os.path.join(curdir, os.pardir))
sys.path.append(pardir)
print(sys.path)
import gymnasium as gym
import gymnasium as gym 
from gymnasium.spaces import *
import numpy as np
from typing import Any, Dict, List, Optional
import logging
from HawkesRLTrading.src.SimulationEntities.TradingAgent import TradingAgent
from HawkesRLTrading.src.SimulationEntities.GymTradingAgent import GymTradingAgent, RandomGymTradingAgent
from HawkesRLTrading.src.Stochastic_Processes.Arrival_Models import ArrivalModel, HawkesArrival
from HawkesRLTrading.src.SimulationEntities.Exchange import Exchange
from HawkesRLTrading.src.Kernel import Kernel
from HawkesRLTrading.src.Envs.HawkesRLTradingEnv import tradingEnv
logger=logging.getLogger(__name__)


['/Users/alex/Desktop/lobSimulationsHawkes/Scripts', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python310.zip', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages', '/Users/alex/Desktop/lobSimulationsHawkes']


Feed in Kwargs arguments: customise trading environment

In [2]:
kwargs={
                "TradingAgent": [],
                "GymTradingAgent": [{"cash": 10000, 
                                    "strategy": "Random",
                                    "action_freq": 2,
                                    "rewardpenalty": 0.4,
                                    "Inventory": {"XYZ": 1000},
                                    "log_to_file": True,
                                    "cashlimit": 1000000}],
                "Exchange": {"symbol": "XYZ",
                "ticksize":0.01,
                "LOBlevels": 2,
                "numOrdersPerLevel": 10,
                "PriceMid0": 45,
                "spread0": 0.05},
                "Arrival_model": {"name": "Hawkes",
                                  "parameters": {"kernelparams": None, 
                                            "tod": None, 
                                            "Pis": None, 
                                            "beta": None, 
                                            "avgSpread": None, 
                                            "Pi_Q0": None}}

            }

#Run the code!

In [3]:
env=tradingEnv(stop_time=300, seed=1, **kwargs)
print("Initial Observations"+ str(env.getobservations()))
Simstate, observations, termination=env.step(action=None)
logger.debug(f"\nSimstate: {Simstate}\nObservations: {observations}\nTermination: {termination}")
i=0
while Simstate["Done"]==False and termination!=True and i<10:
    logger.debug(f"ENV TERMINATION: {termination}")
    AgentsIDs=[k for k,v in Simstate["Infos"].items() if v==True]
    print(f"Agents with IDs {AgentsIDs} have an action available")
    if len(AgentsIDs)>1:
        raise Exception("Code should be unreachable: Multiple gym agents are not yet implemented")
    agent: GymTradingAgent=env.getAgent(ID=AgentsIDs[0])
    assert isinstance(agent, GymTradingAgent), "Agent with action should be a GymTradingAgent"
    action=(agent.id, agent.get_action(data=observations))   
    print(f"Limit Order Book: {observations['LOB0']}")
    print(f"Action: {action}")
    Simstate, observations, termination=env.step(action=action) 
    logger.debug(f"\nSimstate: {Simstate}\nObservations: {observations}\nTermination: {termination}")
    i+=1
    print(f"ACTION DONE{i}")
%reset -f

2024-09-11 14:40:32,124 - HawkesRLTrading.src.SimulationEntities.Exchange - DEBUG - Exchange prices initialized -- Askprice: 45.02, Bidprice: 44.98, Spread: 0.05, MidPrice: 45
2024-09-11 14:40:32,129 - HawkesRLTrading.src.Stochastic_Processes.Arrival_Models - INFO - Missing HawkesArrival model parameters: kernelparams, tod, Pis, beta, avgSpread, Pi_Q0. 
Assuming default values
2024-09-11 14:40:32,129 - HawkesRLTrading.src.SimulationEntities.Exchange - DEBUG - Arrival_model of Exchange specified as HawkesArrival
2024-09-11 14:40:32,182 - HawkesRLTrading.src.SimulationEntities.Exchange - DEBUG - Stock Exchange initalized
2024-09-11 14:40:32,183 - HawkesRLTrading.src.Kernel - DEBUG - Kernel Initialized
2024-09-11 14:40:32,183 - HawkesRLTrading.src.SimulationEntities.TradingAgent - DEBUG - RandomGymTradingAgent 1 requesting kernel wakeup at time 2
2024-09-11 14:40:32,183 - HawkesRLTrading.src.Kernel - DEBUG - Agent current times when being set: {1: 2}
2024-09-11 14:40:32,184 - HawkesRLTrad

Agent IDs: [1]
Exchange ID: 2
first qsize: 100
first qsize: 46
first qsize: 1115
first qsize: 1000
Current_time: 0 
LOB: LOB: 
Ask_L2: 45.03, (46.0, [10.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0])
Ask_L1: 45.02, (100.0, [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0])
Bid_L1: 44.98, (1115.0, [116.0, 111.0, 111.0, 111.0, 111.0, 111.0, 111.0, 111.0, 111.0, 111.0])
Bid_L2: 44.97, (1000.0, [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0])
Spread: 0.04000000000000625
Current Kernel queue: 
[]
Initial Observations{'LOB0': {'Ask_L2': (45.03, 46.0), 'Ask_L1': (45.02, 100.0), 'Bid_L1': (44.98, 1115.0), 'Bid_L2': (44.97, 1000.0)}, 'Cash': 10000, 'Inventory': 1000, 'Positions': {'Ask_L1': [], 'Ask_L2': [], 'Bid_L1': [], 'Bid_L2': []}}
First action time is: 2 seconds
S: 1.2220225587440319
Candidate: 1.2220225587440319
Pointcount is: 1
Arrival Model time series [(1.2220225587440319, 5)]
Before processing order: 

LOB: 
Ask_L2: 45.03, (46.0, [10.0, 4.0, 4.0, 4

InvalidActionError: Agent 1 cannot cancel orders without placing any orders